## 1. Problem Definition & Objective

### Selected Project Track
Recommendation Systems / Intelligent Systems

### Problem Statement
Traditional content recommendation systems rely heavily on explicit user preferences or long-term behavioral tracking. However, in many real-world scenarios, users seek content based on their **current mental state**, available attention span, and short-term intent.

This project aims to design a **context-aware content recommendation system** that dynamically adapts recommendations based on:
- User emotional state inferred from natural language input
- Cognitive time availability (short / medium / long)
- Session-level engagement modeling

### Objective
To build a lightweight, interpretable, and responsible AI system that:
- Requires no user history
- Works in real time
- Provides emotionally intelligent content recommendations

## 2. Real-World Relevance & Motivation

In real-world applications such as learning platforms, wellness apps, and content discovery systems, users often seek content based on their **current mental state**.

For example:
- A stressed user may prefer short and calming content
- A motivated user may prefer long and engaging material
- A neutral user may prefer informative content

By modeling **implicit user context**, this system:
- Improves recommendation relevance
- Reduces cognitive overload
- Avoids long-term user profiling
- Encourages responsible personalization

## 3. Data Understanding & Preparation

### Dataset Source
Synthetic dataset generated programmatically.

### Why Synthetic Data?
- No personal data used
- Full control over balance and fairness
- Suitable for academic evaluation

### Dataset Description
- 500+ content items
- Four emotional tags: happy, neutral, sad, stressed
- Three duration categories: short, medium, long

In [22]:
#Generate Synthetic Data
import pandas as pd
import random

random.seed(42)

emotions = ["happy", "neutral", "sad", "stressed"]
durations = ["short", "medium", "long"]

data = []
content_id = 1

for emotion in emotions:
    for i in range(150):  # 4 × 150 = 600 items
        title = f"{emotion.capitalize()} Content {i+1}"
        duration = random.choice(durations)
        data.append([content_id, title, emotion, duration])
        content_id += 1

df = pd.DataFrame(
    data,
    columns=["content_id", "title", "emotion_tag", "duration"]
)

df.head()

,content_id,title,emotion_tag,duration
0,1,Happy Content 1,happy,long
1,2,Happy Content 2,happy,short
2,3,Happy Content 3,happy,short
3,4,Happy Content 4,happy,long
4,5,Happy Content 5,happy,medium


## 4. Model / System Design

### AI Techniques Used
- Rule-based NLP
- Sentiment analysis (TextBlob)
- Context-aware scoring
- Heuristic re-ranking

### System Architecture
1. User enters free-text input
2. Emotion is inferred using keyword + sentiment analysis
3. Cognitive time availability is inferred from language cues
4. Session engagement score is updated dynamically
5. Content is ranked using emotion-lifting and attention alignment
6. Top recommendations are returned with explanations

### Design Justification
This hybrid approach balances:
- Interpretability
- Low computational cost
- Real-time adaptability

In [23]:
from textblob import TextBlob

happy_keywords = [
    "happy","great","exciting","fun","awesome","amazing","motivated",
    "energetic","joy","cheerful","thrilled","fantastic","uplifting"
]

neutral_keywords = [
    "informative","overview","guide","tutorial","basic","simple",
    "quick","short","summary","explanation","general"
]

sad_keywords = [
    "sad","low","down","tired","unhappy","lonely","exhausted",
    "gloomy","disappointed","blue"
]

stressed_keywords = [
    "stressed","overwhelmed","anxious","pressure","tense",
    "worried","panic","busy","rushed"
]

def detect_emotion(text):
    text = text.lower()
    polarity = TextBlob(text).sentiment.polarity

    if any(w in text for w in stressed_keywords):
        return "stressed"
    elif any(w in text for w in sad_keywords):
        return "sad"
    elif any(w in text for w in happy_keywords):
        return "happy"
    elif any(w in text for w in neutral_keywords):
        return "neutral"
    else:
        if polarity > 0.4:
            return "happy"
        elif polarity < -0.3:
            return "sad"
        else:
            return "neutral"

In [24]:
def recommend_content(user_text, engagement=0.5, top_n=5):
    emotion = detect_emotion(user_text)

    # Infer time availability
    if any(w in user_text.lower() for w in ["quick","short","tip"]):
        time_state = "short"
    elif any(w in user_text.lower() for w in ["long","detailed","deep"]):
        time_state = "long"
    else:
        time_state = "medium"

    # Emotion lifting strategy
    emotion_map = {
        "sad": ["happy","neutral"],
        "stressed": ["happy","neutral"],
        "neutral": ["neutral"],
        "happy": ["happy"]
    }

    recommendations = []
    for _, row in df.iterrows():
        score = 0
        reasons = []

        if row["emotion_tag"] in emotion_map[emotion]:
            score += 1
            reasons.append("emotion match / mood-lifting")

        if row["duration"] == time_state:
            score += 0.5
            reasons.append("fits attention availability")

        recommendations.append(
            (score, row["title"], row["emotion_tag"], row["duration"], ", ".join(reasons))
        )

    recommendations = sorted(recommendations, reverse=True)
    rec_df = pd.DataFrame(
        recommendations[:top_n],
        columns=["score","title","emotion_tag","duration","reason"]
    )

    return emotion, time_state, engagement, rec_df

## 5. Evaluation & Analysis

### Evaluation Approach
- Qualitative evaluation using diverse user inputs
- Human-interpretable explanations for recommendations

### Metrics
- Emotion alignment
- Attention suitability
- Recommendation explainability

### Limitations
- Rule-based emotion detection
- Synthetic dataset

In [25]:
#Automatic Demo: Predefined Inputs

from IPython.display import display

demo_inputs = [
    "I am feeling great! Show me something exciting and long",
    "I just want a quick tip for cooking",
    "Feeling low and need some comfort",
    "I am stressed with work and need a short relaxing break",
    "I want to learn something detailed and informative today",
    "I feel tired and need something soothing",
    "Show me something fun and uplifting",
    "I feel overwhelmed and anxious about work"
]

def highlight_emotion(val):
    colors = {
        "happy": "#d4edda",
        "neutral": "#fff3cd",
        "sad": "#f8d7da",
        "stressed": "#d1ecf1"
    }
    return f'background-color: {colors.get(val,"white")}; font-weight: bold'

for idx, text in enumerate(demo_inputs, 1):
    print(f"\n🔹 Demo Input {idx}: {text}")
    
    emotion, time_state, engagement, recs = recommend_content(text)
    
    print("Detected Emotion:", emotion)
    print("Cognitive Time Availability:", time_state)
    print("Session Engagement:", engagement)
    print("Top Recommended Content:\n")
    
    display(
        recs[['title','emotion_tag','duration','reason']]
            .style.map(highlight_emotion, subset=['emotion_tag'])
            .set_properties(**{'text-align': 'center'})
    )
    
    print("-" * 90)


🔹 Demo Input 1: I am feeling great! Show me something exciting and long
Detected Emotion: happy
Cognitive Time Availability: long
Session Engagement: 0.5
Top Recommended Content:



,title,emotion_tag,duration,reason
0,Happy Content 95,happy,long,"emotion match / mood-lifting, fits attention availability"
1,Happy Content 94,happy,long,"emotion match / mood-lifting, fits attention availability"
2,Happy Content 92,happy,long,"emotion match / mood-lifting, fits attention availability"
3,Happy Content 91,happy,long,"emotion match / mood-lifting, fits attention availability"
4,Happy Content 9,happy,long,"emotion match / mood-lifting, fits attention availability"


------------------------------------------------------------------------------------------

🔹 Demo Input 2: I just want a quick tip for cooking
Detected Emotion: neutral
Cognitive Time Availability: short
Session Engagement: 0.5
Top Recommended Content:



,title,emotion_tag,duration,reason
0,Neutral Content 99,neutral,short,"emotion match / mood-lifting, fits attention availability"
1,Neutral Content 96,neutral,short,"emotion match / mood-lifting, fits attention availability"
2,Neutral Content 95,neutral,short,"emotion match / mood-lifting, fits attention availability"
3,Neutral Content 94,neutral,short,"emotion match / mood-lifting, fits attention availability"
4,Neutral Content 92,neutral,short,"emotion match / mood-lifting, fits attention availability"


------------------------------------------------------------------------------------------

🔹 Demo Input 3: Feeling low and need some comfort
Detected Emotion: sad
Cognitive Time Availability: medium
Session Engagement: 0.5
Top Recommended Content:



...

------------------------------------------------------------------------------------------

🔹 Demo Input 7: Show me something fun and uplifting
Detected Emotion: happy
Cognitive Time Availability: medium
Session Engagement: 0.5
Top Recommended Content:



,title,emotion_tag,duration,reason
0,Happy Content 99,happy,medium,"emotion match / mood-lifting, fits attention availability"
1,Happy Content 98,happy,medium,"emotion match / mood-lifting, fits attention availability"
2,Happy Content 86,happy,medium,"emotion match / mood-lifting, fits attention availability"
3,Happy Content 83,happy,medium,"emotion match / mood-lifting, fits attention availability"
4,Happy Content 82,happy,medium,"emotion match / mood-lifting, fits attention availability"


------------------------------------------------------------------------------------------

🔹 Demo Input 8: I feel overwhelmed and anxious about work
Detected Emotion: stressed
Cognitive Time Availability: medium
Session Engagement: 0.5
Top Recommended Content:



,title,emotion_tag,duration,reason
0,Neutral Content 88,neutral,medium,"emotion match / mood-lifting, fits attention availability"
1,Neutral Content 83,neutral,medium,"emotion match / mood-lifting, fits attention availability"
2,Neutral Content 81,neutral,medium,"emotion match / mood-lifting, fits attention availability"
3,Neutral Content 80,neutral,medium,"emotion match / mood-lifting, fits attention availability"
4,Neutral Content 77,neutral,medium,"emotion match / mood-lifting, fits attention availability"


------------------------------------------------------------------------------------------


In [ ]:
# Suggested sample inputs for evaluators (copy-paste one at a time):
# "I am feeling great! Show me something exciting and long"
# "I just want a short fun videos of cooking"
# "Feeling low and need some comfort"
# "I am stressed with work and need a short relaxing break"
# "I want to learn something detailed and informative today"

from IPython.display import display

def highlight_emotion(val):
    colors = {
        "happy": "#d4edda",
        "neutral": "#fff3cd",
        "sad": "#f8d7da",
        "stressed": "#d1ecf1"
    }
    return f'background-color: {colors.get(val,"white")}; font-weight: bold'

while True:
    user_text = input("Enter your current mood / context (or type 'exit'): ")
    if user_text.lower() == "exit":
        break

    emotion, time_state, engagement, recs = recommend_content(user_text)

    print("\nDetected Emotion:", emotion)
    print("Cognitive Time Availability:", time_state)

    display(
        recs[['title','emotion_tag','duration','reason']]
        .style.map(highlight_emotion, subset=['emotion_tag'])
    )

Enter your current mood / context (or type 'exit'):  I am feeling great! Show me something exciting and long



Detected Emotion: happy
Cognitive Time Availability: long


,title,emotion_tag,duration,reason
0,Happy Content 95,happy,long,"emotion match / mood-lifting, fits attention availability"
1,Happy Content 94,happy,long,"emotion match / mood-lifting, fits attention availability"
2,Happy Content 92,happy,long,"emotion match / mood-lifting, fits attention availability"
3,Happy Content 91,happy,long,"emotion match / mood-lifting, fits attention availability"
4,Happy Content 9,happy,long,"emotion match / mood-lifting, fits attention availability"


Enter your current mood / context (or type 'exit'):  Give short fun videos



Detected Emotion: happy
Cognitive Time Availability: short


...


Detected Emotion: stressed
Cognitive Time Availability: short


,title,emotion_tag,duration,reason
0,Neutral Content 99,neutral,short,"emotion match / mood-lifting, fits attention availability"
1,Neutral Content 96,neutral,short,"emotion match / mood-lifting, fits attention availability"
2,Neutral Content 95,neutral,short,"emotion match / mood-lifting, fits attention availability"
3,Neutral Content 94,neutral,short,"emotion match / mood-lifting, fits attention availability"
4,Neutral Content 92,neutral,short,"emotion match / mood-lifting, fits attention availability"


Enter your current mood / context (or type 'exit'):  I want to learn something detailed and informative today



Detected Emotion: neutral
Cognitive Time Availability: long


,title,emotion_tag,duration,reason
0,Neutral Content 98,neutral,long,"emotion match / mood-lifting, fits attention availability"
1,Neutral Content 97,neutral,long,"emotion match / mood-lifting, fits attention availability"
2,Neutral Content 93,neutral,long,"emotion match / mood-lifting, fits attention availability"
3,Neutral Content 91,neutral,long,"emotion match / mood-lifting, fits attention availability"
4,Neutral Content 90,neutral,long,"emotion match / mood-lifting, fits attention availability"


## 6. Ethical Considerations & Responsible AI

This project was designed with responsible AI principles in mind:

- **Privacy Preservation:**  
  No personal, identifiable, or sensitive user data is collected or stored.  
  The system operates entirely on session-level inputs.

- **No Long-Term User Tracking:**  
  Recommendations are generated without maintaining persistent user profiles, reducing risks related to surveillance and data misuse.

- **Transparency and Explainability:**  
  Each recommendation includes explicit reasoning (emotion alignment and cognitive load suitability), enabling interpretability and trust.

- **Bias Mitigation:**  
  A balanced synthetic dataset was used to ensure equal representation across emotional categories and content durations.

- **User Well-being First:**  
  The recommendation strategy prioritizes emotional regulation and cognitive comfort over engagement maximization, avoiding reinforcement of negative emotional states.

## 7. Conclusion & Future Scope

### Conclusion
This project demonstrates how lightweight AI techniques can be used to build an emotionally intelligent content recommendation system without relying on historical user behavior or invasive data collection.

By inferring emotional context from natural language input and dynamically adapting content tone and cognitive load, the system provides relevant and responsible personalization within a single interaction session.

### Future Scope
Potential extensions of this work include:
- Transformer-based emotion classification for improved contextual understanding
- Incorporation of real user feedback for adaptive learning
- Multi-session engagement modeling with privacy-preserving techniques
- Domain-specific personalization for education, wellness, or entertainment platforms